<a href="https://colab.research.google.com/github/CrAvila/IA/blob/main/Taller1/Stroke_PM_Front.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

In [6]:
import gradio as gr

# Function to classify inputs from the user
def classify_input(value, config):
    ranges = config["ranges"]
    categories = config["categories"]
    for i, (start, end) in enumerate(ranges):
        if start <= value < end:
            return categories[i]
    return categories[-1]

In [1]:

from random import random

# Events defined in the model
events = {
    "quantitative" : {

        "age": {
            "ranges" : [
                (0,18),
                (18,65),
                (65, float('inf'))
            ],

            "categories" : [
                'Child',
                'Adult',
                'Senior'
            ]
        },

        "avg_glucose_level": {
            "ranges" : [
                (0,80),
                (80,120),
                (120, float('inf'))
            ],

            "categories" : [
                'Low',
                'Normal',
                'High'
            ]
        },

        "bmi": {
            "ranges": [
                (0,18.5),
                (18.5, 25),
                (25, float('inf'))
            ],

            "categories": [
                'Underweight',
                'Normal',
                'Overweight',
                'Unknown'
            ]
        }
    }
}


variables_to_use = [
    'age',
    'avg_glucose_level',
    'bmi',
    'hypertension',
    'heart_disease',
]

def get_probability(model, variables, input_data, events):

    # Categorize the quantitative variables
    def get_from_tree(variable, value, events):
        if isinstance(value, str):
            return value

        if variable in events['quantitative']:
            ranges = events['quantitative'][variable]['ranges']
            categories = events['quantitative'][variable]['categories']
            for i, (lower, upper) in enumerate(ranges):
                if lower <= value < upper:
                    return categories[i]
        return value

    # Create the tuple of tuples for the input
    categorized_input = []
    for variable, value in zip(variables, input_data):
        categorized_value = get_from_tree(variable, value, events)
        categorized_input.append(categorized_value)

    input_tuple = tuple((variable, value) for variable, value in zip(variables,categorized_input))

    # Specific for this model
    key = input_tuple + (('stroke', 1),)

    # Extract probability
    probability = model[len(variables)][key]

    return probability

def get_stroke_risk(model, variables, input_data):
    probability = get_probability(model, variables, input_data, events)
    rgn = random()

    risk = 0

    if rgn < probability or probability > 0.05:
        risk = 1

    return risk, probability

In [4]:
import requests
import ast

# Model is stored in an external Gist file, so we just need to import it.
gist_raw_url = "https://gist.githubusercontent.com/CrAvila/2bb2b00cc37bc4fc7d85b6e8773132b1/raw"

response = requests.get(gist_raw_url)
if response.status_code == 200:
    gist_content = response.text
    gist_content = gist_content.strip()
    model_content = gist_content[len("model="):]

    # Use ast.literal_eval to safely evaluate the content as a dictionary
    try:
        model = ast.literal_eval(model_content)
    except (ValueError, SyntaxError):
        print("Error while evaluating Gist content as a dictionary.")
else:
    print("Failed to fetch Gist content.")

In [8]:
def predict_stroke(age, avg_glucose_average, bmi_value, bmi_known, hypertension, heart_disease):
    # Classify the input features
    age_category = classify_input(age, events["quantitative"]["age"])
    glucose_category = classify_input(avg_glucose_average, events["quantitative"]["avg_glucose_level"])

    bmi_config = events["quantitative"]["bmi"]
    bmi_category = "Unknown" if bmi_known else classify_input(bmi_value, bmi_config)
    hypertension = 1 if hypertension else 0
    heart_disease = 1 if heart_disease else 0

    inputs = [age_category, glucose_category, bmi_category, hypertension, heart_disease]
    r, p = get_stroke_risk(model, variables_to_use, inputs)
    pf = "{:.2%}".format(p)

    # Your prediction logic here
    prediction_result = f"You are{' ' if r == 1 else ' not '}at risk of a stroke."
    desc = f"{pf} of people with these characteristics suffered a stroke."

    return prediction_result, desc

with gr.Blocks() as app:
    age = gr.Number(label="Age")
    avg_glucose_average = gr.Number(label="Average Glucose")
    bmi_value = gr.Number(label="Body Mass Index", default=None)
    bmi_known = gr.Checkbox(label="I don't know my BMI")
    hypertension = gr.Checkbox(label="Hypertension")
    heart_disease = gr.Checkbox(label="Heart Disease")

    output = [gr.Textbox(label="Stroke Prediction"),
              gr.Textbox(label="Description")]

    predict_btn = gr.Button("Predict")
    predict_btn.click(fn=predict_stroke, inputs=[age, avg_glucose_average, bmi_value, bmi_known, hypertension, heart_disease], outputs=output)

app.launch(share=True)

<ipython-input-8-dcc042c880e7>:24: GradioUnusedKwargWarning: You have unused kwarg parameters in Number, please remove them: {'default': None}
  bmi_value = gr.Number(label="Body Mass Index", default=None)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0456000fc5022f84da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
